In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 25.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPTNeoForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from transformers import Trainer, TrainingArguments
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/depression_detection/Suicide_Detection_new")
df.sample(5)

,Unnamed: 0,text,class
168298,252479,anyone down to chat? I'm really awake rn and b...,non-suicide
230127,345143,"Guys I have a question Listen, will you marry me😳",non-suicide
583,870,I swear to god if one more person tells me to ...,suicide
54168,81228,"ight, i just pissed in my eye on accident, and...",non-suicide
135212,203001,I’m out of antidepressants but can’t bring mys...,suicide


In [ ]:
df.columns

Index(['Unnamed: 0', 'text', 'class'], dtype='object')

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df.rename(columns={'text': 'clean_text', 'class': 'is_depression'}, inplace=True)

In [ ]:
df['is_depression'] = df['is_depression'].replace({'suicide': 1, 'non-suicide': 0})

In [ ]:
df.sample(5)

,clean_text,is_depression
20285,I'm done posting here. I'm OD'ing tonight.I'm ...,1
122839,23 and hopelessI'm new to reddit so in sorry i...,1
190542,Feeling like a bit of a stopgapI'm married wit...,1
87811,I convince myself that I am just overdramatic ...,1
8492,My StoryI was born in the fall of '89 a bastar...,1


In [ ]:
df=df.sample(7500)

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [ ]:
import torch
from transformers import GPTNeoForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader

In [ ]:
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification

# Define model and tokenizer
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='', eos_token='', pad_token='')
model = GPTNeoForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Convert DataFrame to list of tuples
train_dataset = [(row['clean_text'], row['is_depression']) for _, row in train_df.iterrows()]
test_dataset = [(row['clean_text'], row['is_depression']) for _, row in test_df.iterrows()]

# Function to generate prompt for each text sample
def generate_prompt(data_point):
    return f"Analyse the following text and predict if it expresses signs of depression or not. Return 0 for no depression and 1 for depression.\n\n{data_point[0]}"

# Add prompt to each sample in the dataset
train_dataset = [(generate_prompt(data), label) for data, label in train_dataset]
test_dataset = [(generate_prompt(data), label) for data, label in test_dataset]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, config)

In [ ]:
class DepressionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        encoding = {key: val.squeeze() for key, val in encoding.items()}
        encoding['labels'] = torch.tensor(label, dtype=torch.long)
        return encoding



In [ ]:
# Prepare datasets
train_dataset = DepressionDataset(train_df['clean_text'].tolist(), train_df['is_depression'].tolist(), tokenizer)
test_dataset = DepressionDataset(test_df['clean_text'].tolist(), test_df['is_depression'].tolist(), tokenizer)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Low batch size due to potential memory constraints
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install -U transformers accelerate


In [ ]:
!pip install accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.31.0
    Uninstalling accelerate-0.31.0:
      Successfully uninstalled accelerate-0.31.0


In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)

        # Ensure logits shape is compatible with the loss function
        if isinstance(outputs, tuple):
            logits = outputs[0]  # If model returns tuple, take the first element
        else:
            logits = outputs.logits

        # Compute loss
        loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.023200,0.280707
2,0.000000,0.146658
3,0.618100,0.138117


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=18000, training_loss=0.2771310832738769, metrics={'train_runtime': 1297.5933, 'train_samples_per_second': 13.872, 'train_steps_per_second': 13.872, 'total_flos': 1177491603456000.0, 'train_loss': 0.2771310832738769, 'epoch': 3.0})

In [ ]:
import numpy as np

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

model.eval()
all_labels = []
all_preds = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

eval_dataloader = DataLoader(test_dataset, batch_size=1)
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    labels = batch.pop('labels').to(device)

    with torch.no_grad():
        outputs = model(**batch)

    # Determine the correct way to access logits based on the model's output
    if isinstance(outputs, tuple):  # If model returns a tuple
        logits = outputs[0]  # Take the first element of the tuple
    else:
        logits = outputs.logits

    # Apply softmax to get probabilities
    probs = torch.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)

    all_labels.extend(labels.cpu().numpy())
    all_preds.extend(preds.cpu().numpy())

# Print additional diagnostics
print("Unique labels:", np.unique(all_labels))
print("Predictions distribution:", np.bincount(all_preds))
print("Label distribution:", np.bincount(all_labels))

# Print individual predictions for further inspection (optional)
print("Sample predictions:", all_preds[:10])
print("Sample labels:", all_labels[:10])

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
try:
    f1 = f1_score(all_labels, all_preds, average='binary', pos_label=1)
except ValueError as e:
    print("Error computing F1 score:", e)
    f1 = f1_score(all_labels, all_preds, average='macro')  # Fallback to macro average

# Display confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Unique labels: [0 1]
Predictions distribution: [753 747]
Label distribution: [737 763]
Sample predictions: [0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
Sample labels: [0, 1, 1, 0, 1, 0, 1, 1, 1, 1]
Confusion Matrix:
 [[725  12]
 [ 28 735]]
Accuracy: 0.9733333333333334
F1 Score: 0.9735099337748344
